In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
import pandas as pd

# Define launch sites with their coordinates
launch_sites_df = pd.DataFrame({
    'Site': ['CCAFS LC-40', 'KSC LC-39A', 'VAFB SLC-4E', 'CCAFS SLC-40'],
    'Latitude': [28.5623, 28.6084, 34.6328, 28.5632],
    'Longitude': [-80.5774, -80.6043, -120.6108, -80.5768]
})


In [8]:
# Create a map centered at the US
spacex_map = folium.Map(location=[28.5623, -80.5774], zoom_start=5)

# Add MarkerCluster
marker_cluster = MarkerCluster().add_to(spacex_map)

# Add launch site markers with labels
for idx, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Site'],
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')  # rocket icon if font-awesome is available
    ).add_to(marker_cluster)
    
    # Add DivIcon label next to the marker
    folium.map.Marker(
        [row['Latitude'], row['Longitude']],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color: black">{row["Site"]}</div>',
        )
    ).add_to(spacex_map)

# Display map
spacex_map

In [9]:
spacex_map.save("map.html")


In [19]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [20]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [21]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [22]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [23]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [25]:
# Task 2
# Define colors for success/failure
colors = {1: 'green', 0: 'red'}

# Initialize the map (centered at NASA JSC or any location)
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterate through all launches in spacex_df
for idx, row in spacex_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=500,  # radius in meters
        color=colors[row['class']],  # green for success, red for failure
        fill=True,
        fill_color=colors[row['class']],
        fill_opacity=0.6,
        popup=(f"Launch Site: {row['Launch Site']}<br>"
               f"Outcome: {'Success' if row['class']==1 else 'Failure'}")
    ).add_to(site_map)

# Display the map
site_map


In [26]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [27]:
marker_cluster = MarkerCluster()

In [28]:
# Create a new column 'marker_color' based on 'class' values
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Check the last 10 rows to confirm
spacex_df.tail(10)


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [29]:
# Add all launches to the MarkerCluster
for index, record in spacex_df.iterrows():
    # Create a Marker for each launch
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=(f"Launch Site: {record['Launch Site']}<br>"
               f"Outcome: {'Success' if record['class']==1 else 'Failure'}"),
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    # Add marker to the cluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


In [30]:
spacex_map.save("map.html")

In [31]:
from math import radians, cos, sin, asin, sqrt


In [32]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in km between two points
    on the earth specified in decimal degrees.
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in km
    return c * r


In [33]:
# Create a new column 'Distance_to_NASA_km'
launch_sites_df['Distance_to_NASA_km'] = launch_sites_df.apply(
    lambda row: haversine(row['Long'], row['Lat'], nasa_coordinate[1], nasa_coordinate[0]),
    axis=1
)

launch_sites_df


,Launch Site,Lat,Long,Distance_to_NASA_km
0,CCAFS LC-40,28.562302,-80.577356,1413.328592
1,CCAFS SLC-40,28.563197,-80.576820,1413.366602
2,KSC LC-39A,28.573255,-80.646895,1406.434271
3,VAFB SLC-4E,34.632834,-120.610745,2462.753721


In [34]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [35]:
spacex_map.save("map.html")

In [36]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [37]:
# Launch site coordinates (CCAFS SLC-40)
launch_site_lat = 28.5632
launch_site_lon = -80.5768

# Closest coastline coordinates (obtained via MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163


In [38]:
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance from launch site to coastline: {distance_coastline:.2f} km")


Distance from launch site to coastline: 0.51 km


In [39]:
from folium.features import DivIcon

# Coordinate of coastline point
coastline_coordinate = [coastline_lat, coastline_lon]

# Create a marker with DivIcon showing distance
distance_marker = folium.Marker(
    location=coastline_coordinate,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)

# Optionally, add a small circle to highlight the coastline point
folium.Circle(
    location=coastline_coordinate,
    radius=200, 
    color='blue',
    fill=True,
    fill_opacity=0.5
).add_to(site_map)

# Display the map
site_map


In [40]:
# Coordinates of launch site and coastline
launch_site_coordinate = [launch_site_lat, launch_site_lon]
coastline_coordinate = [coastline_lat, coastline_lon]

# Create a PolyLine between launch site and coastline
lines = folium.PolyLine(
    locations=[launch_site_coordinate, coastline_coordinate],
    color='red',   # line color
    weight=2,      # line thickness
    dash_array='5,5'  # dashed line (optional)
)

# Add the line to the map
site_map.add_child(lines)

# Display the map
site_map
